In [56]:
import random

def read_tsp_file(filename):
    """TSP Lib形式のファイルを読み込む"""
    with open(filename, 'r') as file:
        lines = file.readlines()
    
    nodes = []
    in_node_section = False
    for line in lines:
        if "NODE_COORD_SECTION" in line:
            in_node_section = True
            continue
        if "EOF" in line:
            break
        if in_node_section:
            parts = line.strip().split()
            if len(parts) >= 3:
                nodes.append((int(parts[0]), float(parts[1]), float(parts[2])))
    return nodes

def write_tsp_file(filename, nodes):
    """TSP Lib形式のファイルを書き出す"""
    with open(filename, 'w') as file:
        file.write("NAME : Generated_TSP_Instance\n")
        file.write("TYPE : TSP\n")
        file.write("DIMENSION : {}\n".format(len(nodes)))
        file.write("NODE_COORD_SECTION\n")
        for node in nodes:
            file.write("{} {} {}\n".format(node[0], node[1], node[2]))
        file.write("EOF\n")

def create_similar_tsp_instance(original_nodes, dataset_nodes, replace_ratio=0.01, seed=None, mapping_filename="mapping.txt"):
    """
    元のTSPインスタンスの一部のノードを置き換えて類似インスタンスを作成し、対応表を出力する。
    
    Parameters:
    - original_nodes: 元のTSPインスタンスのノードリスト。
    - dataset_nodes: 全ノードを含むデータセット。
    - replace_ratio: 置き換える割合（デフォルトは10%）。
    - seed: 乱数のシード値（再現性確保のためのオプション）。
    - mapping_filename: 対応表を保存するファイル名。
    
    Returns:
    - new_instance: 置き換え後のTSPインスタンス。
    """
    if seed is not None:
        random.seed(seed)
    
    num_replace = int(len(original_nodes) * replace_ratio)
    if len(dataset_nodes) - len(original_nodes) < num_replace:
        raise ValueError("元のインスタンスに含まれないノードが不足しています。")
    
    replace_nodes = random.sample(original_nodes, num_replace)
    available_nodes = list(set(dataset_nodes) - set(original_nodes))
    new_nodes = random.sample(available_nodes, num_replace)
    
    new_instance = original_nodes[:]
    mapping = []
    
    for i, (old_node, new_node) in enumerate(zip(replace_nodes, new_nodes), start=1):
        new_node_id = 900000 + i  # 変更後のノードIDを9000番台に設定
        new_instance[new_instance.index(old_node)] = (new_node_id, new_node[1], new_node[2])
        mapping.append((old_node[0], new_node_id))  # (変更前ID, 変更後ID)
    
    # 対応表をファイルに出力
    with open(mapping_filename, 'w') as file:
        file.write("Original_ID -> New_ID\n")
        for old_id, new_id in mapping:
            file.write(f"{old_id} -> {new_id}\n")
    
    return new_instance

# ノートブックでの計算
seed_value = 46
dataset_nodes = read_tsp_file("mona-lisa100K.tsp")

num_original_nodes = 90000
random.seed(seed_value)
original_nodes_sample = random.sample(dataset_nodes, num_original_nodes)
original_nodes = [(i + 1, node[1], node[2]) for i, node in enumerate(original_nodes_sample)]
write_tsp_file("original_90k_n5.tsp", original_nodes)

similar_nodes = create_similar_tsp_instance(original_nodes, dataset_nodes, replace_ratio=0.01, seed=seed_value, mapping_filename="mappin90k_n5.txt")
write_tsp_file("similar_90k_n5.tsp", similar_nodes)
